To switch to dbt profile with the `arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo` role that I created

In [ ]:
nano ~/.aws/config

# In nano add this profile
[profile dbt-glue]
role_arn = arn:aws:iam::026090528544:role/GlueInteractiveSessionClientRole-dbt-CLI-demo
source_profile = Joshua_Harris_ETL
region = ap-southeast-2

# switch profile for the terminal session
export AWS_PROFILE=dbt-glue

# check your cureent profile
aws sts get-caller-identity